<a href="https://colab.research.google.com/github/ftempesta/Data-Science-Online/blob/master/Laboratorio_2_Clasificaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Indicaciones

- Trabaja en grupos de 2 o 3 personas.
- Haz una copia del notebook en `File -> Save a copy in Drive`.
- Para hacer la entrega, descarga el notebook en `File -> Download -> Download .ipynb`

## 1. Árbol de decisión

Descargamos el dataset de animales:

In [ ]:
import pandas as pd

df = pd.read_csv('http://www.cse.msu.edu/~ptan/dmbook/tutorials/tutorial6/vertebrate.csv')
df

Como son muy pocos datos (N=14) y varias clases, convertiremos este problema (clasificación "multi-clase") en una tarea de clasificación binaria (sólo dos clases). En este caso, lo haremos transformando las etiquetas de la clase de forma de tener sólo dos instancias: mamíferos y no-mamíferos.

In [ ]:
df['Class'] = df['Class'].replace(['fishes','birds','amphibians','reptiles'],'non-mammals')
df

Con `crosstab` podemos examinar la relación entre los atributos `Warm-blooded` y `Gives birth` con respecto a la clase (equivalente a `table` de R).

In [ ]:
pd.crosstab([df['Warm-blooded'], df['Gives Birth']], df['Class'])

Ahora entrenaremos un árbol de decisión y lo graficaremos:

In [ ]:
from sklearn import tree

Y = df['Class']
X = df.drop(['Name','Class'], axis=1)

clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3)
clf = clf.fit(X, Y)

In [ ]:
import pydotplus 
from IPython.display import Image

dot_data = tree.export_graphviz(clf, 
                                feature_names=X.columns, 
                                class_names=['mammals','non-mammals'], 
                                filled=True, 
                                out_file=None) 
graph = pydotplus.graph_from_dot_data(dot_data) 
Image(graph.create_png())

Probaremos el clasificador con datos fabricados por nosotros:

In [ ]:
testData = [['gila monster',0,0,0,0,1,1,'non-mammals'],
           ['platypus',1,0,0,0,1,1,'mammals'],
           ['owl',1,0,0,1,1,0,'non-mammals'],
           ['dolphin',1,1,1,0,0,0,'mammals']]

testData = pd.DataFrame(testData, columns=df.columns)
testData

,Name,Warm-blooded,Gives Birth,Aquatic Creature,Aerial Creature,Has Legs,Hibernates,Class
0,gila monster,0,0,0,0,1,1,non-mammals
1,platypus,1,0,0,0,1,1,mammals
2,owl,1,0,0,1,1,0,non-mammals
3,dolphin,1,1,1,0,0,0,mammals


In [ ]:
X_test = testData.drop(['Name', 'Class'], axis=1)
y_test = testData['Class']

y_pred = clf.predict(X_test)
predictions = pd.concat([testData['Name'], pd.Series(y_pred, name='Predicted Class')], axis=1)
predictions


,Name,Predicted Class
0,gila monster,non-mammals
1,platypus,non-mammals
2,owl,non-mammals
3,dolphin,mammals


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

**Pregunta 1**

¿En qué instancia(s) se equivocó el clasificador en los datos de prueba que le pasamos?

---

## 2. Overfitting

El código de abajo genera N=1500 puntos en dos dimensiones, con dos clases del mismo tamaño:

- La clase 1 es generada con una mezcla de tres distribuciones normales en dos dimensiones (mezcla de Gaussianas), centradas en las coordenadas $(6,14), (10,6)$ y $(14, 14)$.
- La clase 0 es generada con una distribución uniforme en el cuadrado de lado 20 con una esquina en el origen.

El código genera un gráfico donde se pueden ver las instancias. Los puntos de la clase 1 está en rojo mientras que los de la clase 0 en negro.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import random

%matplotlib inline

N = 1500

mean1 = [6, 14]
mean2 = [10, 6]
mean3 = [14, 14]
cov = [[3.5, 0], [0, 3.5]]  # covarianza

np.random.seed(50)
X = np.random.multivariate_normal(mean1, cov, int(N/6))
X = np.concatenate((X, np.random.multivariate_normal(mean2, cov, int(N/6))))
X = np.concatenate((X, np.random.multivariate_normal(mean3, cov, int(N/6))))
X = np.concatenate((X, 20*np.random.rand(int(N/2),2)))
Y = np.concatenate((np.ones(int(N/2)),np.zeros(int(N/2))))

plt.plot(X[:int(N/2),0],X[:int(N/2),1],'r+',X[int(N/2):,0],X[int(N/2):,1],'k.',ms=4)

En el siguiente código hacemos holdout del 20% para test.
Graficamos el accuracy de distintos árboles de decisión, variando el parámetro `max_depth`, que dice qué tan profundo puede ser el árbol (cuántos niveles puede tener como mucho).

In [ ]:
#########################################
# Training and Test set creation
#########################################

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.8, random_state=1)

from sklearn import tree
from sklearn.metrics import accuracy_score

#########################################
# Model fitting and evaluation
#########################################

maxdepths = [2,3,4,5,6,7,8,9,10,15,20,25,30,35,40,45,50]

trainAcc = np.zeros(len(maxdepths))
testAcc = np.zeros(len(maxdepths))

index = 0
for depth in maxdepths:
    clf = tree.DecisionTreeClassifier(max_depth=depth)
    clf = clf.fit(X_train, Y_train)
    Y_predTrain = clf.predict(X_train)
    Y_predTest = clf.predict(X_test)
    trainAcc[index] = accuracy_score(Y_train, Y_predTrain)
    testAcc[index] = accuracy_score(Y_test, Y_predTest)
    index += 1
    
#########################################
# Plot of training and test accuracies
#########################################
    
plt.plot(maxdepths,trainAcc,'ro-',maxdepths,testAcc,'bv--')
plt.legend(['Accuracy en training set','Accuracy en test set'])
plt.xlabel('Parámetro max_depth')
plt.ylabel('Accuracy')

**Pregunta 2**

Describa el gráfico resultante. ¿Cuál consideraría que es un buen valor para el parámetro `max_depth`? ¿Por qué?

---

## 3. KNN

En este clasificador, la etiqueta (clase) de una instancia se predice basado en las $k$ instancias más cercanas en el dataset de entrenamiento. En este caso, $k$ es un hiperparámetro que debe ser dado por el usuario junto con la métrica de distancia.

El siguiente código aplica el clasificador KNN con distintos valores de $k$ sobre los datos generados en el punto anterior:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
%matplotlib inline

numNeighbors = [1, 5, 10, 15, 20, 25, 30]
trainAcc = []
testAcc = []

for k in numNeighbors:
    clf = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
    clf.fit(X_train, Y_train)
    Y_predTrain = clf.predict(X_train)
    Y_predTest = clf.predict(X_test)
    trainAcc.append(accuracy_score(Y_train, Y_predTrain))
    testAcc.append(accuracy_score(Y_test, Y_predTest))

plt.plot(numNeighbors, trainAcc, 'ro-', numNeighbors, testAcc,'bv--')
plt.legend(['Accuracy en training set','Accuracy en test set'])
plt.xlabel('Número de vecinos k')
plt.ylabel('Accuracy')

**Pregunta 3**

Compare el gráfico de KNN con el de Árboles de Decisión.

¿Cuál clasificador (algoritmo de aprendizaje) considera que es mejor para estos datos? ¿Por qué funciona mejor ese clasificador en este caso?

---

## 4. Support Vector Machine Classifier (SVC)

Entrenaremos un SVM sobre los mismos datos.

Vimos rápidamente en clase que SVC tiene un parámetro de *regularización* que regula cuánto puede "equivocarse" el modelo al separar los datos con un hiperplano (una recta o un plano son tipos de hiperplanos) en dos clases. Esto le permite tolerar puntos mal clasificados, y poder buscar un hiperplano que separe las dos clases lo mejor posible.

El siguiente código entrena un SVM cambiando el parámetro $C$.

In [ ]:
from sklearn.svm import SVC

C = [0.01, 0.1, 0.2, 0.5, 0.8, 1, 5, 10, 20, 50]
SVMtrainAcc = []
SVMtestAcc = []

for param in C:
    clf = SVC(C=param, kernel='linear')
    clf.fit(X_train, Y_train)
    Y_predTrain = clf.predict(X_train)
    Y_predTest = clf.predict(X_test)
    SVMtrainAcc.append(accuracy_score(Y_train, Y_predTrain))
    SVMtestAcc.append(accuracy_score(Y_test, Y_predTest))

plt.plot(C, SVMtrainAcc, 'ro-', C, SVMtestAcc, 'bv--')
plt.legend(['Accuracy en training set','Accuracy en test set'])
plt.xlabel('C')
plt.xscale('log')
plt.ylabel('Accuracy')


**Pregunta 4**

¿Por qué SVM tiene tan mal rendimiento, comparado con Árboles de Decisión y KNN?

---

Ahora, haremos lo mismo, pero usaremos un kernel no lineal. Esto puede interpretarse como si aplicáramos una transformación a los datos para que sean linealmente separables:

In [ ]:
from sklearn.svm import SVC

C = [0.01, 0.1, 0.2, 0.5, 0.8, 1, 5, 10, 20, 50]
SVMtrainAcc = []
SVMtestAcc = []

for param in C:
    clf = SVC(C=param,kernel='rbf',gamma='auto')
    clf.fit(X_train, Y_train)
    Y_predTrain = clf.predict(X_train)
    Y_predTest = clf.predict(X_test)
    SVMtrainAcc.append(accuracy_score(Y_train, Y_predTrain))
    SVMtestAcc.append(accuracy_score(Y_test, Y_predTest))

plt.plot(C, SVMtrainAcc, 'ro-', C, SVMtestAcc,'bv--')
plt.legend(['Accuracy en training set','Accuracy en test set'])
plt.xlabel('C')
plt.xscale('log')
plt.ylabel('Accuracy')

## 5. Ensemble Classifiers (Random Forests)

Random Forest corresponde a una categoría de modelos llamada "ensemble classifiers", donde se "ensamblan" distintos clasificadores y se usan en conjunto para tomar una decisión. Existen otros modelos o métodos, como Bagging (Bootstrap aggregating) o AdaBoost.

En términos simples, estos métodos entrenan muchos árboles de decisión, cada uno con una muestra distinta de datos de entrenamiento, y a la hora de clasificar, "somete a votación" los resultados de cada uno de los árboles.

El siguiente código entrena tres modelos de ensamblaje, donde cada modelo usa como base un Árbol de Decisión con `max_depth=10`. 

Cada modelo usa 500 árboles.

El gráfico resultante muestra el accuracy en el conjunto de entrenamiento (izquierda) y en el conjunto de testing (derecha).

In [ ]:
from sklearn import ensemble
from sklearn.tree import DecisionTreeClassifier

numBaseClassifiers = 500
maxdepth = 10


trainAcc = []
testAcc = []

clf = ensemble.RandomForestClassifier(n_estimators=numBaseClassifiers)
clf.fit(X_train, Y_train)
Y_predTrain = clf.predict(X_train)
Y_predTest = clf.predict(X_test)
trainAcc.append(accuracy_score(Y_train, Y_predTrain))
testAcc.append(accuracy_score(Y_test, Y_predTest))

clf = ensemble.BaggingClassifier(DecisionTreeClassifier(max_depth=maxdepth),n_estimators=numBaseClassifiers)
clf.fit(X_train, Y_train)
Y_predTrain = clf.predict(X_train)
Y_predTest = clf.predict(X_test)
trainAcc.append(accuracy_score(Y_train, Y_predTrain))
testAcc.append(accuracy_score(Y_test, Y_predTest))

clf = ensemble.AdaBoostClassifier(DecisionTreeClassifier(max_depth=maxdepth),n_estimators=numBaseClassifiers)
clf.fit(X_train, Y_train)
Y_predTrain = clf.predict(X_train)
Y_predTest = clf.predict(X_test)
trainAcc.append(accuracy_score(Y_train, Y_predTrain))
testAcc.append(accuracy_score(Y_test, Y_predTest))

methods = ['Random Forest', 'Bagging', 'AdaBoost']
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
ax1.bar([1.5,2.5,3.5], trainAcc)
ax1.set_xticks([1.5,2.5,3.5])
ax1.set_xticklabels(methods)

ax2.bar([1.5,2.5,3.5], testAcc)
ax2.set_xticks([1.5,2.5,3.5])
ax2.set_xticklabels(methods)

**Pregunta 5**

¿Cómo se compara el resultado de estos clasificadores con los árboles de decisión vistos más arriba?

Discuta con sus compañer@s: ¿por qué cree que "promediar" el resultado de muchos árboles de decisión sea mejor que basarse en los resultados de uno solo?

---

fin